# Regression Multifeature


### Data Preparation

Data preparation: Now keeps only tail data and normalizes HiggsM values for conditioning.

Model: ConditionalMLP now accepts multiple inputs based on the number of features:

The HiggsM conditioning information is projected and added to the network, allowing the model to learn how the velocity field varies with HiggsM.

Training: Now passes the corresponding HiggsM value for each sample during training.

Sampling (cell 12): Samples across the entire HiggsM range including the pole region. The model uses learned patterns from tail data to extrapolate predictions into the pole region where it has no direct training data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

raw_df = pd.read_hdf('data_files/new_Input_NonResonant_yy_25th_January2026.h5', key='VBF_Polarisation_Tree')

# Define bounds and number of bins
lower_bound = 115000
upper_bound = 135000
num_bins = 1100

# Creating frequency distribution for HiggsM
freq = pd.cut(raw_df['HiggsM'], bins=num_bins)
freq_counts = freq.value_counts().sort_index()
higgsM_values = [interval.mid for interval in freq_counts.index]
frequencies = freq_counts.values.tolist()

# Create DataFrame for ML Model
# Applying the bounds to HiggsM values
model_df = pd.DataFrame({'HiggsM': higgsM_values, 'Frequency': frequencies})
mask1 = model_df['HiggsM'] > upper_bound
mask2 = model_df['HiggsM'] < lower_bound
model_df_tails = model_df[mask1 | mask2]

# Applying the same bounds to the raw data for the other features
grouped_df = raw_df.groupby(pd.cut(raw_df['HiggsM'], bins=num_bins), observed=True).mean(numeric_only=True)
grouped_df.index.name = 'HiggsM_interval'
grouped_df = grouped_df.reset_index()
grouped_df['HiggsM_interval'] = grouped_df['HiggsM_interval'].apply(lambda x: x.mid).astype(float)
mask3 = grouped_df['HiggsM_interval'] > upper_bound
mask4 = grouped_df['HiggsM_interval'] < lower_bound
others_df_tails = grouped_df[mask3 | mask4]

# Combine tail data for other features and HiggsM frequency into a single DataFrame for training
model_df_tails.index.name = others_df_tails.index.name
combined_tails_df = pd.concat([model_df_tails, others_df_tails.drop(columns=['HiggsM', 'HiggsM_interval'])], axis=1)

# Save the combined tails DataFrame to a CSV file for inspection
# Filter out unnecessary columns and save the relevant features along with HiggsM and Frequency
combined_tails_df.to_csv('data_dump/combined_tails_data.csv')
features = ['DNN_score', 'Frequency', 'DPhi_jj', 'Eta_jj', 'M_jj', 'Njets', 'OO1']
features_array = combined_tails_df[features].to_numpy()
num_features = features_array.shape[1]
# print(features_array.shape)
# print(features_array[:5, :])

# Array of HiggsM values for training
HiggsM_array = combined_tails_df['HiggsM'].to_numpy()



In [2]:
from sklearn.preprocessing import MinMaxScaler


# Normalise function
# Un-nomralise later using sampled_points_original = scaler.inverse_transform(sampled_points_normalized)
scaler_features = MinMaxScaler()
scaler_HiggsM = MinMaxScaler()

# Normalise features and HiggsM separately
features_normalised = scaler_features.fit_transform(features_array)
HiggsM_normalised = scaler_HiggsM.fit_transform(HiggsM_array.reshape(-1, 1))

### Random noise data points to start with, this is the initial distribution p0(x0), original code used random noise, replace with the Higgs tail data.


## Model

### The multi-feature model has input channels based on the number of features

In [ ]:
## Apply model

### Taking output data and comparing it to true data by saving both as a csv file

In [ ]:
# Denormalising the values produced by the model
HiggsM_denorm = scaler_HiggsM.inverse_transform(HiggsM_sample_norm.reshape(-1, 1)).flatten()
features_denorm = scaler_features.inverse_transform(xt.detach().cpu().numpy())

pred_points = pd.DataFrame({'HiggsM': HiggsM_denorm,
                            'DNN_score': features_denorm[:, 0],
                            'Frequency': features_denorm[:, 1],
                            'DPhi_jj': features_denorm[:, 2],
                            'Eta_jj': features_denorm[:, 3], 
                            'M_jj': features_denorm[:, 4],
                            'Njets': features_denorm[:, 5],
                            'OO1': features_denorm[:, 6],
                            })

# Constructing the pole region data for both the predicted and true data
model_df_pole = model_df[~(mask1 | mask2)]
others_df_pole = grouped_df[~(mask3 | mask4)]
model_df_pole.index.name = others_df_pole.index.name
true_points = pd.concat([model_df_pole, others_df_pole.drop(columns=['HiggsM', 'HiggsM_interval'])], axis=1)
variables = ['HiggsM', 'DNN_score', 'Frequency', 'DPhi_jj', 'Eta_jj', 'M_jj', 'Njets', 'OO1']
true_points = true_points[variables]
lower_bound = 115000
upper_bound = 135000
pred_points_pole = pred_points[(pred_points['HiggsM'] >= lower_bound) & (pred_points['HiggsM'] <= upper_bound)]

# Saving data to csv for comparison
csv1 = pd.DataFrame(pred_points_pole, columns=['HiggsM', 'DNN_score', 'Frequency', 'DPhi_jj', 'Eta_jj', 'M_jj', 'Njets', 'OO1'])
csv2 = pd.DataFrame(true_points, columns=['HiggsM', 'DNN_score', 'Frequency', 'DPhi_jj', 'Eta_jj', 'M_jj', 'Njets', 'OO1'])
csv1.to_csv('data_dump/multi_predicted_pole_region.csv', index=False)
csv2.to_csv('data_dump/multi_true_pole_region.csv', index=False)



## KL Divergence calculation to assess quality of the model


In [ ]:
# Ensuring that both arrays have the same shape for KL Divergence calculation by trimming both ends
counter = 0
if pred_points_pole.shape > true_points.shape:
    while pred_points_pole.shape != true_points.shape:
        if counter % 2 == 0: 
            pred_points_pole = np.delete(pred_points_pole, -1, axis=0)
            counter += 1
        else:
            pred_points_pole = np.delete(pred_points_pole, 0, axis=0)
        counter += 1

elif pred_points_pole.shape < true_points.shape:
    while pred_points_pole.shape != true_points.shape:
        if counter % 2 == 0: 
            true_points = np.delete(true_points, -1, axis=0)
            counter += 1
        else:
            true_points = np.delete(true_points, 0, axis=0)
            counter += 1
        

# Convert true_points to a numpy array, pred_points_pole is already a numpy array
true_points_array = true_points.to_numpy()
print(true_points_array.shape, pred_points_pole.shape)

# Implement KL Divergence calculation
for i in range(len(features)):
    true_sum = np.sum(true_points_array[:, i])
    p = np.array(true_points_array[:, i]) / true_sum
    pred_sum = np.sum(pred_points_pole[:, i])
    q = np.array(pred_points_pole[:, i]) / pred_sum

    epsilon = 1e-10
    p_smooth = p + epsilon
    q_smooth = q + epsilon

    kl_divergence = np.sum(p_smooth * np.log(p_smooth / q_smooth))
    
    
    if np.isinf(kl_divergence) or np.isnan(kl_divergence):
        error_df = pd.DataFrame(p_smooth/q_smooth)
        error_df.to_csv('data_dump/check.csv', index=False)


    if i==3:
        error_df = pd.DataFrame(p_smooth/q_smooth)
        error_df.to_csv('data_dump/check1.csv', index=False)


    print(f"KL Divergence between true and predicted distributions in the pole region: {kl_divergence} for feature {features[i]}")






(400, 8) (400, 8)
KL Divergence between true and predicted distributions in the pole region: 9.022945566141522e-09 for feature DNN_score
KL Divergence between true and predicted distributions in the pole region: 0.00011006198634116359 for feature Frequency
KL Divergence between true and predicted distributions in the pole region: 0.001026079114787892 for feature DPhi_jj
KL Divergence between true and predicted distributions in the pole region: 1.6471510407342e-05 for feature Eta_jj
KL Divergence between true and predicted distributions in the pole region: 1.2014802021659376e-05 for feature M_jj
KL Divergence between true and predicted distributions in the pole region: 8.364150537747126e-05 for feature Njets
KL Divergence between true and predicted distributions in the pole region: 1.9099076413029158e-05 for feature OO1
